# Simulation for token market equilibrium
deterministic version, log utility

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import solara
import plotly.graph_objects as go

In [2]:
# Canvas
P_MAX = 50
Q_MAX = 50
p_ax = np.linspace(0.01,P_MAX,1000)
q_ax = np.linspace(0.01,Q_MAX,1000)

In [3]:
# Variables
e_u = solara.reactive(100)
beta = solara.reactive(0.98)
m = solara.reactive(10)
p_h = solara.reactive(1)
phi_b = solara.reactive(0.025)
phi_v = solara.reactive(0.025)
h = solara.reactive(0.01)
theta = solara.reactive(0.08)
r = solara.reactive(0.04)
p_1 = solara.reactive(1.2)
M = solara.reactive(10)
e_v = solara.reactive(100)

In [4]:
@solara.component
def Page():
    # Sliders
    solara.FloatSlider("e_u", value=e_u, min=0, max=200, step=10)
    solara.FloatSlider("e_v", value=e_v, min=0, max=200, step=10)
    solara.FloatSlider("beta", value=beta, min=0, max=1, step=0.01)
    solara.FloatSlider("m", value=m, min=0, max=50, step=10)
    solara.FloatSlider("M", value=M, min=0, max=50, step=10)
    solara.FloatSlider("p_h", value=p_h, min=-2, max=2)
    solara.FloatSlider("phi_b", value=phi_b, min=0, max=1, step=0.01)
    solara.FloatSlider("phi_v", value=phi_v, min=0, max=1, step=0.01)
    solara.FloatSlider("h", value=h, min=0, max=1, step=0.01)
    solara.FloatSlider("theta", value=theta, min=0, max=1, step=0.01)
    solara.FloatSlider("r", value=r, min=0, max=1, step=0.01)
    solara.FloatSlider("p(t+1)", value=p_1, min=-2, max=2)

    # compute variables
    b = beta.value/(1+beta.value)
    A1 = (beta.value*e_u.value - (1+beta.value)*p_h.value)
    A2 = (1+b*phi_v.value)
    A = A1*A2
    B = - b*e_v.value
    C = beta.value*(1+phi_b.value+phi_v.value)
    D = (1-h.value)*(1+theta.value)*p_1.value/(1+r.value)
    
    # inverse demand curve
    def Q_d(p):
        Q = A1/ p / (C+D/p)
        return Q

    # inverse supply curve
    def Q_s(p):
        Q = (M.value + B/p) / A2
        return Q

    # plot points
    Qs = Q_s(p_ax)
    Qd = Q_d(p_ax)

    # equilibrium price (analytical)
    p_eqm = 2*B*D/(A-M.value*D-B*C-np.sqrt((M.value*D+B*C-A)**2-4*M.value*B*C*D))

    # reset function
    def reset():
        try:
            e_u.set(100)
            beta.set(0.98)
            m.set(10)
            p_h.set(1)
            phi_b.set(0.025)
            phi_v.set(0.025)
            h.set(0.01)
            theta.set(0.08)
            r.set(0.04)
            p_1.set(1.2)
            M.set(10)
            e_v.set(100)
        except RuntimeWarning:
            pass
    with solara.Row():
        solara.Button("Reset", on_click=reset)

    # figure using Plotly
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=Qs, y=p_ax, mode='lines', name='Token Supply'))
    fig.add_trace(go.Scatter(x=Qd, y=p_ax, mode='lines', name='Token Demand'))
    fig.update_xaxes(title="Quantity", range=[0, Q_MAX])
    fig.update_yaxes(title="Price", range=[0, P_MAX])
    with solara.Card():
        solara.FigurePlotly(fig)

In [5]:
Page()

Cannot show ipywidgets in text